In [1]:
from numba import cuda
reset_d = cuda.get_current_device()
reset_d.reset()

In [ ]:
import os
class myconfig():
    data_path = r"/home/work/lib_data/KISTI/summary_data_ctgr.pkl"
    access_token = os.getenv('HUGGINGFACE_TOKEN')
    model = "KIML/llama3_8B_hjy_v2"
    cache_dir = r"/home/work/lib_data/Llama3"
mcfg = myconfig()

In [3]:
import pandas as pd
summary_data = pd.read_pickle(mcfg.data_path)
title = list(summary_data["title"])
summary = list(summary_data["summary"])

In [4]:
import torch
torch.cuda.empty_cache()
torch.cuda.is_available()

True

In [5]:
from huggingface_hub import login
login(mcfg.access_token)

/home/work/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/work/.cache/huggingface/token
Login successful


# 1. 모델 불러오기

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
import numpy as np

tokenizer = AutoTokenizer.from_pretrained(mcfg.model, cache_dir=mcfg.cache_dir)
model = AutoModelForCausalLM.from_pretrained(mcfg.model, cache_dir=mcfg.cache_dir)

Loading checkpoint shards: 100%|██████████| 4/4 [00:19<00:00,  4.86s/it]


In [8]:
model.to(device)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (n

# 2. 프롬프트 생성

In [ ]:
system_prompt = """You are a helpful AI assistant for financial and commercial analysis."""
user_prompt = f"""If customer does not make late payments in October 2005, generate the values ​​of the following conditions:

Amount of given credit in NT dollars: 
Gender: 
Education: 
Marital status: 
Age: 
Repayment status in June, 2005: 
Repayment status in May, 2005: 
Repayment status in April, 2005: 
Amount of bill statement in June, 2005: 
Amount of bill statement in May, 2005: 
Amount of bill statement in April, 2005: 
Amount of previous payment in June, 2005: 
Amount of previous payment in May, 2005: 
Amount of previous payment in April, 2005: 

Do not create repeated content or multiple conditions.
"""
    
base_prompt = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>{user_prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"

input = tokenizer(base_prompt, return_tensors="pt").to(device)

# input = tokenizer(user_prompt, return_tensors="pt").to(device)

# 3. 추론(generation)

In [115]:
# from transformers import pipeline

# pipe = pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     torch_dtype=torch.float16,
#     device=device,
#     )

In [12]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [154]:
results = []

In [ ]:
for i in range(2000):
    generated_outputs = model.generate(**input,
                                    max_new_tokens = 200,
                                # return_dict_in_generate=True
    )

    # generated_outputs.sequences = input+output
    input_length = 1 if model.config.is_encoder_decoder else input.input_ids.shape[1]

    generated_tokens = generated_outputs[:,input_length:]

    sentence = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)
    
    results.append(sentence)

    gc.collect()
    torch.cuda.empty_cache()

In [156]:
len(results)

2000

In [157]:
result_no_late = pd.DataFrame(results)

In [158]:
result_no_late.to_csv("./result_no_late.csv")

In [9]:
results_2 = []

In [29]:
system_prompt = """You are a helpful AI assistant for financial and commercial analysis."""
user_prompt = f"""If customer will default on his/her credit card payment in October 2005, fill the [] with values ​​of the following conditions:

Amount of given credit in NT dollars: []
Gender: []
Education: [] 
Marital status: []
Age: []
Repayment status in June, 2005: []
Repayment status in May, 2005: []
Repayment status in April, 2005: []
Amount of bill statement in June, 2005: []
Amount of bill statement in May, 2005: []
Amount of bill statement in April, 2005: []
Amount of previous payment in June, 2005: []
Amount of previous payment in May, 2005: []
Amount of previous payment in April, 2005: []

Do not generate repeated content or multiple conditions.
"""
    
base_prompt = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>{user_prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"

input = tokenizer(base_prompt, return_tensors="pt").to(device)

# input = tokenizer(user_prompt, return_tensors="pt").to(device)

In [ ]:
for i in range(2000):
    generated_outputs = model.generate(**input,
                                    max_new_tokens = 250,
                                # return_dict_in_generate=True
    )

    # generated_outputs.sequences = input+output
    input_length = 1 if model.config.is_encoder_decoder else input.input_ids.shape[1]

    generated_tokens = generated_outputs[:,input_length:]

    sentence = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)
    
    print(sentence)
    
    results_2.append(sentence)

    gc.collect()
    torch.cuda.empty_cache()

In [32]:
len(results_2)

2042

In [33]:
result_late = pd.DataFrame(results_2)
result_late.to_csv("./result_late.csv")